# Setup Guide for MD Simulations

## Identifying Your Target PDB File

Selecting the right PDB file is a crucial first step in structure-based drug design. The choice should align with the specific goals of your research. When navigating the Protein Data Bank, which houses a myriad of protein structures, consider the following to ensure the relevance and accuracy of your simulation:

### Considerations for Selecting a PDB File

#### Species Relevance:
- Preferably choose a structure derived from the species that your research is targeting, typically human. This ensures that the sequences and structural features are representative of your biological system of interest.

#### Research Alignment:
- If your study involves correlating computational data with experimental data, such as binding kinetics of enzyme activity assays, it's crucial that the protein structure reflects the system used in the lab.

#### Ligand Consideration:
- The presence and relevance of a bound ligand can significantly impact your research. If your study focuses on a particular ligand or a specific binding site, ensure that the PDB file includes a structure with the ligand bound to the correct site.

For this exercise, we will choose T4-Lysozyme, bound with a toluene (as ligand).


<div class="alert alert-block alert-info">
<b>Tip:</b> You will need Biopython , NGLVIEW and an account on SWISS-MODEL for this module
</div>

## Step 1 : Cleaning Up Your PDB File

Once you've selected a suitable PDB file, the next step is to clean it up for simulation:

- Remove any water molecules, ligands, or ions that are not of interest or that might interfere with the simulation.
- Check for and repair any missing atoms or residues that are critical for the structure.
- Ensure that the protein structure is in the correct conformational state for your study.


In [1]:
from Bio.PDB import *
import nglview as nv
from Bio import *

In [2]:
# Specify the PDB code
pdb_code = "4w53"
# Assuming the .pdb file is in the current working directory
pdb_filename = f"{pdb_code}.pdb"

# Initialize the parser
parser = PDBParser()

# Parse the structure
structure = parser.get_structure(pdb_code, pdb_filename)
print(f"Structure {pdb_code} loaded successfully!")

Structure 4w53 loaded successfully!


In [3]:
# Visualize the structure with NGLView
view = nv.show_biopython(structure)
view

NGLWidget()

### Conclusion of above view:
As observed from the above view, it is clear that the protein is bound to two different ligands. However, EPE is a solvent use in crystallization condition of the protein, and is not of interest to us. We only want to keep the Toulene molecule, as that is our ligand.

The following function automatically identifies the ligands as well as engineered and mutated residues, and allows us to keep the ligand of choice. See the summary and type "MBN", the pdb ID of toulene in this protein.

In [4]:
def fetch_ligands_and_mutations(pdb_filename):
    parser = PDB.PDBParser()
    structure = parser.get_structure('PDB', pdb_filename)
    
    ligands = set()
    mutations = set()
    
    for model in structure:
        for chain in model:
            for residue in chain:
                # Check if residue is a heteroatom (ligand)
                if residue.id[0] != ' ':
                    ligands.add(residue.resname)
                # Check for annotations in the residue id (This part is simplified)
                if 'MUTA' in residue.id[0] or 'ENGINEE' in residue.id[0]:
                    mutations.add(residue)
    
    print(f"Found {len(ligands)} ligands: {ligands}")
    print(f"Found {len(mutations)} mutations or engineered residues.")
    
    # Prompt for user input on ligands to keep
    ligands_to_keep = input("Enter the names of the ligands you want to keep, separated by a comma (e.g., MBN,EPE): ")
    ligands_to_keep = [ligand.strip().upper() for ligand in ligands_to_keep.split(',')]
    
    return ligands_to_keep

# Example usage
ligands_to_keep = fetch_ligands_and_mutations(pdb_filename)
print("Ligands to keep:", ligands_to_keep)

Found 3 ligands: {'EPE', 'MBN', 'HOH'}
Found 0 mutations or engineered residues.
Enter the names of the ligands you want to keep, separated by a comma (e.g., MBN,EPE): HOH,MBN
Ligands to keep: ['HOH', 'MBN']


### Keep chosen ligand in your protein

class LigandSelect(Select):
    def accept_residue(self, residue):
        # Retain only MBN ligand and the rest of the protein
        if residue.get_resname() == ligands_to_keep:
            return 0
        else:
            return 1
io = PDBIO()
io.set_structure(structure)
modified_structure = f"{pdb_code}_mod.pdb"
io.save(modified_structure, LigandSelect())

### Further Processing / Homology Model

It is possible to have missing residues and loops from your structure. To fill in such gaps, we need to homology model our protein. 
While there are tools available such as MODELLER (requires academic or organizational affiliation), SWISS-MODEL, is a web interface which allows any user to model free of cost. Notably, there are also [multiple open source python based tools](https://github.com/topics/homology-modeling) which allow a user to do that. 

For the context of automation,I will call SWISS-MODEL using an API, to perform this task.

In [6]:
import requests
import json
from Bio import Seq

#### Step 1: Fetch the Uniprot ID to first fetch the full sequence
UNIPROT ID is generally provided in the webpage of the pdb ID. Since, published pdbs might have a few residues missing, the full uniprot sequnce ensures that such gaps/missing residues will be filled/modelled in while homology modelling.

In [8]:
def fetch_uniprot_id(pdb_id):
    url = f"https://data.rcsb.org/rest/v1/core/polymer_entity/{pdb_id}/1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for reference in data.get("rcsb_polymer_entity_container_identifiers", {}).get("reference_sequence_identifiers", []):
            if reference.get("database_name") == "UniProt":
                return reference.get("database_accession")
    return "UniProt ID not found"

# Example usage
pdb_id = "4W53"  # Replace with your PDB ID
uniprot_id = fetch_uniprot_id(pdb_id)
print(f"UniProt ID for PDB {pdb_id}: {uniprot_id}")

UniProt ID for PDB 4W53: P00720


#### Step 2: Fetch the full sequence using the UNIPROT ID

In [9]:
def fetch_uniprot_sequence(uniprot_id):
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print("Failed to fetch data")
        return None

# Example: Fetching the sequence for a UniProt ID. Replace 'P00720' with UniProt ID for your portein
sequence = fetch_uniprot_sequence(uniprot_id)  
print(sequence) 

>sp|P00720|ENLYS_BPT4 Endolysin OS=Enterobacteria phage T4 OX=10665 GN=E PE=1 SV=2
MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNCNGVITK
DEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRCALINMVFQMGETGVAGFTNSLRM
LQQKRWDEAAVNLAKSIWYNQTPNRAKRVITTFRTGTWDAYKNL



#### Step 3 : Get your SWISS-MODEL API authentication token

In [10]:
# Define the URL for the SWISS-MODEL API endpoint
# Endpoint for the API to authenticate
auth_url = 'https://swissmodel.expasy.org/api-token-auth/'

# Your SWISS-MODEL username and password
credentials = {
    'username': 'chatterjeepayal242@gmail.com',
    'password': 'DyxHk7'
}

# Make the POST request to get the token
response = requests.post(auth_url, data=credentials)

if response.status_code == 200:
    # The request was successful, extract the token
    token = response.json().get('token')
    print('Your authentication token is:', token)
else:
    # There was an error
    print('Error submitting job:', response.status_code)
    print(response.text)

Your authentication token is: d50c507f0961f6c5d828c489fb350a1dc17e6323


#### Step 4 : Use your modified pdb with the ligand "MBN" and crystal water as the template in homology modeling. 

Note : If you are interested in learning further about homology modelling, SWISS-MODEL has a nice [documentation](https://swissmodel.expasy.org/docs/examples) along with examples and assessment of the quality of models.

In the following code, replace token with your token.

In [11]:
with open("4w53_mod.pdb") as f:
    template_coordinates = f.read()

    
# The base URL for SWISS-MODEL's API
base_url = "https://swissmodel.expasy.org"

# The endpoint for the automodel function
usertemplate_endpoint = "/user_template"

# Your API token
api_token = token ### REPLACE HERE WITH YOUR OWN TOKEN

# The headers including your authorization token
headers = {
    "Authorization": f"Token {api_token}"
}

# The data you want to send in the POST request
data = {
    "target_sequences": ["MNIFEMLRIDERLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNCNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRCALINMVFQMGETGVAGFTNSLRMLQQKRWDEAAVNLAKSIWYNQTPNRAKRVITTFRTGTWDAYKNL"],
    "template_coordinates": template_coordinates,
    "project_title":"CADDTOOLS Homology Model"
}

# Make the POST request to the automodel endpoint
response = requests.post(f"{base_url}{usertemplate_endpoint}", headers=headers, json=data)

# Check if the request was successful
if response.status_code == 200:
    print("Job submitted successfully.")
    # You can process the response here
else:
    print(f"Error submitting job: {response.status_code}")
    print(response.text)

Job submitted successfully.


#### Step 4: Fetch the results/status

In [12]:
# Obtain the project_id from the response created above
project_id = response.json()["project_id"]

# And loop until the project completes
import time
while True:
    # We wait for some time
    time.sleep(10)

    # Update the status from the server 
    response = requests.get(
        f"https://swissmodel.expasy.org/project/{ project_id }/models/summary/", 
        headers={ "Authorization": f"Token {token}" })

    # Update the status
    status = response.json()["status"]

    print('Job status is now', status)

    if status in ["COMPLETED", "FAILED"]:
        break

Job status is now COMPLETED


#### Step 5: Check if the job is COMPLETED and fetch the model coordinates

Once you receieve the url, click on it to directly download.

In [13]:
response_object = response.json()
if response_object['status']=='COMPLETED':
    for model in response_object['models']:
        print(model['coordinates_url'])

https://swissmodel.expasy.org/project/bdb0fc/models/01.pdb.gz


<div class="alert alert-block alert-info">
<b>Tip:</b> Note that this API is not a bypass of the multiple validation files you receive when using the website directly, but only an attempt to automate system prepartion process. 
</div>